In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import twokenize
from nltk.tokenize import TweetTokenizer
from FeatureFunctions import getfeatures


In [12]:
datafile = "datasets/train/SemEval2018-T3-train-taskA.txt"
datafile2 = "datasets/train/SemEval2018-T3-train-taskA_emoji_ironyHashtags.txt"
trainingdata = pd.read_csv(datafile, sep = "\t", header=0)

type(trainingdata['Tweet text'])
trainingdata = trainingdata[['Label','Tweet text']]
train_tweets = trainingdata['Tweet text']

testfile = 'datasets/goldtest_TaskA/SemEval2018-T3_gold_test_taskA_emoji.txt'
testdata = pd.read_csv(testfile, sep="\t", header=0)
testdata = testdata[['Label','Tweet text']]


In [ ]:
training_features = getfeatures.get(trainingdata, 'Tweet text')
test_features = getfeatures.get(testdata, 'Tweet text')
# training_features

In [ ]:
# ### Write to file

# outputfolder = input("Output folder: ")
# outputname = input("Output name + extension: ")
# trainingdata.to_csv(outputfolder + "/" + outputname, sep='\t')

In [ ]:
### Import the classifier
import sys
sys.path.insert(0, 'libsvm')

from svmutil import *

In [ ]:
### Create X and Y
import sklearn

## Binary + scalar features
x = training_features[['PunctuationFlood', 'CharFlood', 'CapitalizedCount', 'HashtagCount', 'Hashtag2WordRatio', 'TweetCharLength', 'TweetWordLength', 'EmojiCount', 'FinalPunctuation']].values

## Vector features - ToDo
# x = trainingdata['CharFourgramVector'].values

y = trainingdata['Label'].tolist()

In [ ]:
### Train and get train error

prob  = svm_problem(y, x)
param = svm_parameter('-t 2')
m = svm_train(prob, param)
p_label, p_acc, p_val = svm_predict(y, x, m)
ACC, MSE, SCC = evaluations(y, p_label)

In [ ]:
test_x = test_features[['PunctuationFlood', 'CharFlood', 'CapitalizedCount', 'HashtagCount', 'Hashtag2WordRatio', 'TweetCharLength', 'TweetWordLength', 'EmojiCount', 'FinalPunctuation']].values
test_y = test_features['Label'].tolist()